# Explore Individual columns for preliminary insights 

### Import packages

In [17]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import base64

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# FIPS codes

### Loading FIPS Codes

In [39]:
with open('data/FipsCode.csv') as f:
    fips=pd.read_csv(f, delimiter=',')

fips = fips.rename(columns={'name': 'County', 'state': 'State'})
    
fips.head(5)
fips_full = fips
fips_full.head(5)

fips          County State
0     0   UNITED STATES   NaN
1  1000         ALABAMA   NaN
2  1001  Autauga County    AL
3  1003  Baldwin County    AL
4  1005  Barbour County    AL

### Cleaning FIPS 

In [19]:
#Fix inconsistencies
fips['County'] = fips['County'].str.title()
fips['County'] = fips['County'].str.replace('County','')
fips['County'] = fips['County'].str.replace('Borough','')
fips['County'] = fips['County'].str.replace('Parish','')
fips['County'] = fips['County'].str.replace('Census Area','')
fips['County'] = fips['County'].str.replace('Municipality','')
fips['County'] = fips['County'].str.replace('City','')
fips['County'] = fips['County'].str.replace('Gr$','Gra')
fips['County'] = fips['County'].str.replace('Isl$','Island')
fips['County'] = fips['County'].str.replace('Island','')
fips['County'] = fips['County'].str.replace('Desoto','De Soto')
fips['County'] = fips['County'].str.replace('Do̱A Ana','Dona Ana')
fips['County'] = fips['County'].str.replace("'S",'s')

fips['County'] = fips['County'].str.strip()
fips['County'] = fips['County'].str.replace('And$','')
fips['County'] = fips['County'].str.replace('Cumberl$','Cumberland')
fips['County'] = fips['County'].str.strip()
fips.loc[(fips['State'] == 'MD') & (fips['fips'] == 24005), 'County'] = 'Baltimore County'
fips.loc[(fips['State'] == 'MD') & (fips['fips'] ==  24510), 'County'] = 'Baltimore City'

#Drop fips that are NaN
fips.loc[fips['State'].isnull(),'State_NaN'] = 'Yes'
fips.loc[fips['State'].notnull(), 'State_NaN'] = 'No'
fips = fips[fips['State_NaN'] == 'No']
fips = fips.drop(columns = ['State_NaN'])

fips.head(5)

fips   County State
2  1001  Autauga    AL
3  1003  Baldwin    AL
4  1005  Barbour    AL
5  1007     Bibb    AL
6  1009   Blount    AL

## Zip code 

### Load Zip code data

In [20]:
with open('data/ZipCodes.csv') as f:
    zips=pd.read_csv(f, delimiter=',')

#Clean zip code data
zips = zips[["state_abbr",'county',"zipcode"]]
zips = zips.rename(columns={'county': 'County_zip','state_abbr':'State_zip'})
zips['County_zip'] = zips['County_zip'].str.title()
zips['State_zip'] = zips['State_zip'].str.strip()
zips['County_zip'] = zips['County_zip'].str.strip()

zips.head(0)

Empty DataFrame
Columns: [State_zip, County_zip, zipcode]
Index: []

## Provider data

Description: This dataset will be used to calculate the number of vaccination sites per county per state. Note that this includes testing and vaccination sites through October 15th, 2021. 

### Read in Provider data

In [21]:
with open('data/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv') as f:
    df_full=pd.read_csv(f, delimiter=',')

#Subset data and rename columns
df=df_full[["county","State","offers_vaccine",'status','fulladdr']]
df = df.rename(columns={'county': 'County'})

#extract postal codes
df['postal_code'] = df['fulladdr'].str.extract(r'(\d{5})')


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Cleaning provider data

In [22]:
#Strip the white spaces off States
df['County'] = df['County'].str.title()
df['State'] = df['State'].str.strip()
df['County'] = df['County'].str.strip()


#Remove cases that are closed or don't offer vaccinations 
df = df[df['status'] == 'Open']
df = df[df['offers_vaccine'] == 'Yes']

#Remove cases that are territories of the US 
df = df[df['State'] != 'PR']
df = df[df['State'] != 'GU']
df = df[df['State'] != 'VI']
df = df[df['State'] != 'AS']

#Identify NaN's  (There are three)
df.loc[df['State'].isnull(),'State_NaN'] = 'Yes'
df.loc[df['State'].notnull(), 'State_NaN'] = 'No'
#df['County'].isnull().sum()/ len(df) *100


# Replace NaN with approproate state based on Full Address variable
df.loc[(df['County'] == 'Dallas') & (df['State_NaN'] == 'Yes'), 'State'] = 'TX'  
df.loc[(df['County'] == 'San Diego') & (df['State_NaN'] == 'Yes'), 'State'] = 'CA'  
df.loc[(df['County'] == 'Cumberland') & (df['State_NaN'] == 'Yes'), 'State'] = 'ME' 

#Replace spelling errors in counties
df.loc[(df['County'] == 'King') & (df['State'] == 'CA'), 'County'] = 'Kings'  
df.loc[(df['County'] == 'Bel Air') & (df['State'] == 'MD'), 'County'] = 'Harford'  
df.loc[(df['County'] == 'Cambridge') & (df['State'] == 'MD'), 'County'] = 'Dorchester'  
df.loc[(df['County'] == 'Catonsville') & (df['State'] == 'MD'), 'County'] = 'Baltimore'  
df.loc[(df['County'] == 'Johnson') & (df['State'] == 'CA'), 'County'] = 'Ventura'  
df.loc[(df['County'] == 'Suwanee') & (df['State'] == 'FL'), 'County'] = 'Suwannee'  
df.loc[(df['County'] == 'Dewitt') & (df['State'] == 'IL'), 'County'] = 'De Witt'  
df.loc[(df['County'] == 'Oswego') & (df['State'] == 'IL'), 'County'] = 'Kendall'  
df.loc[(df['County'] == 'La Salle') & (df['State'] == 'IL'), 'County'] = 'Lasalle'  
df.loc[(df['County'] == 'Hagerstown') & (df['State'] == 'MD'), 'County'] = 'Washington'  
df.loc[(df['County'] == 'Chestertown') & (df['State'] == 'MD'), 'County'] = 'Kent'  
df.loc[(df['County'] == 'Cheverly') | (df['County'] == 'Clinton') & (df['State'] == 'MD'), 'County'] = 'Prince Georges'  
df.loc[(df['County'] == 'Easton') & (df['State'] == 'MD'), 'County'] = 'Talbot'  
df.loc[(df['County'] == 'Glen Burnie') & (df['State'] == 'MD'), 'County'] = 'Anne Arundel'  
df.loc[(df['County'] == 'La Plata') & (df['State'] == 'MD'), 'County'] = 'Charles'  
df.loc[(df['County'] == 'Towson') & (df['State'] == 'MD'), 'County'] = 'Baltimore'  
df.loc[(df['County'] == 'Lutherville-Timonium') & (df['State'] == 'MD'), 'County'] = 'Baltimore County'  
df.loc[(df['County'] == 'Olney') & (df['State'] == 'MD'), 'County'] = 'Montgomery'  
df.loc[(df['County'] == 'Somerset') & (df['State'] == 'MA'), 'County'] = 'Plymouth'  
df.loc[(df['County'] == 'Hartford') & (df['State'] == 'MA'), 'County'] = 'Hampden'  
df.loc[(df['County'] == 'Pearl') & (df['State'] == 'MS'), 'County'] = 'Pearl River'  
df.loc[(df['County'] == 'Consolidated Municipality Of Carson City') & (df['State'] == 'NV'), 'County'] = 'Carson'  
df.loc[(df['County'] == 'Middlesex') & (df['State'] == 'NY'), 'County'] = 'New York'  
df.loc[(df['County'] == 'Wilkin') & (df['State'] == 'ND'), 'State'] = 'MN'  
df.loc[(df['County'] == 'Mecklenburg') & (df['State'] == 'SC'), 'State'] = 'NC'  
df.loc[(df['County'] == 'Whitfield') & (df['State'] == 'TN'), 'State'] = 'GA'  
df.loc[(df['County'] == 'Texas') & (df['State'] == 'TX'), 'County'] = 'El Paso'
df.loc[(df['County'] == 'New Boston') & (df['State'] == 'TX'), 'County'] = 'Bowie'
df.loc[(df['County'] == 'Clark And Cowlitz') & (df['State'] == 'WA'), 'County'] = 'Cowlitz'
df.loc[(df['County'] == 'Trumnull') & (df['State'] == 'OH'), 'County'] = 'Trumbull'  
df.loc[(df['County'] == 'Caldwell') & (df['State'] == 'OH'), 'County'] = 'Noble'  


#Identify county NaN's  (There are 1198)
df.loc[df['County'].isnull(),'County_NaN'] = 'Yes'
df.loc[df['County'].notnull(), 'County_NaN'] = 'No'


#Subset data that is not NaN
df = df[df['County_NaN'] == 'No']

#zip_USA = zips[(zips['State_zip']== 'MD') & (zips['County_zip'] != 'Baltimore City') & (zips['County_zip'] != 'Baltimore County')]


### Cleaning data - continued

In [23]:
#Replace acronyms to be consistent
df['State'] = df['State'].str.replace('Fl','FL')

#Remove words like 'County' from county name to be consistent 
df['County'] = df['County'].str.replace('Baltimore County','Baltimore Co')
df['County'] = df['County'].str.replace('Baltimore City','Baltimore Ci')
df['County'] = df['County'].str.replace('County','')
df['County'] = df['County'].str.replace('City','')
df['County'] = df['County'].str.replace("'S",'s')
df['County'] = df['County'].str.replace('Baltimore Co','Baltimore County')
df['County'] = df['County'].str.replace('Baltimore Ci','Baltimore City')
df['County'] = df['County'].str.replace('Seat','')
df['County'] = df['County'].str.replace('Parish','')
df['County'] = df['County'].str.replace('Island','')
df['County'] = df['County'].str.replace('Saint','St.')
df['County'] = df['County'].str.replace('St Lucie','St. Lucie')
df['County'] = df['County'].str.replace('Sacremento','Sacramento')
df['County'] = df['County'].str.replace('^Ca$','Riverside')
df['County'] = df['County'].str.replace('San Fr$','San Francisco')
df['County'] = df['County'].str.replace('Duvall','Duval')
df['County'] = df['County'].str.replace('Idian','Indian')
df['County'] = df['County'].str.replace('Desoto','De Soto')
df['County'] = df['County'].str.replace('Doña Ana','Dona Ana')


#Strip white spaces off counties
df['County'] = df['County'].str.strip()

df['County'] = df['County'].str.replace('^La$','Los Angeles')
df.loc[(df['County'] == 'Lasalle') & (df['State'] == 'LA'), 'County'] = 'La Salle'  
#df['County'] = df['County'].str.replace('Of','of')
#df['County'] = df['County'].str.replace('Dekalb','DeKalb')

zip_balt = zips[(zips['County_zip']== "Baltimore City") & (zips['State_zip']== "MD")]['zipcode']
df.loc[(df['County'] == 'Baltimore') & (df['State'] == 'MD'), 'County'] = 'Baltimore County'  
df.loc[df['postal_code'].isin(zip_balt), 'County'] = 'Baltimore City'  

#zip_MD = zips[(zips['State_zip']== 'MD') & (zips['County_zip'] != 'Baltimore City') & (zips['County_zip'] != 'Baltimore County')]
#df.loc[df['postal_code'].isin(zip_MD['zipcode']) & df['State'] == 'MD', 'County'] = zip_MD['County_zip']  




In [24]:

#Group
df_sites = df.groupby(['State','County']).count()

#Reset index 
df_sites = df_sites[['offers_vaccine']].reset_index()
df_sites

State          County  offers_vaccine
0       AK  Aleutians East               6
1       AK  Aleutians West               2
2       AK       Anchorage              38
3       AK          Bethel               1
4       AK     Bristol Bay               1
...    ...             ...             ...
2557    WY      Sweetwater               6
2558    WY           Teton               3
2559    WY           Uinta               5
2560    WY        Washakie               2
2561    WY          Weston               2

[2562 rows x 3 columns]

In [25]:
#df2 = df_sites[df_sites["State"] == 'NJ']
#print(df2.to_string())

### Sites by county and state

Group the number of sites that are open and offer vaccinations by state and county

In [26]:
#fips2 = fips[fips["State"] == 'ME']
#print(fips2.to_string())

In [33]:
VacSites_1 = df_sites.merge(fips, left_on=['State','County'], right_on=['State','County'])

VacSites_1 = VacSites_1[['County','State','fips','offers_vaccine']]
VacSites_1 = VacSites_1.rename(columns={'fips': 'FIPS','offers_vaccine':'Sites'})
VacSites_1



County State   FIPS  Sites
0     Aleutians East    AK   2013      6
1     Aleutians West    AK   2016      2
2          Anchorage    AK   2020     38
3             Bethel    AK   2050      1
4        Bristol Bay    AK   2060      1
...              ...   ...    ...    ...
2561      Sweetwater    WY  56037      6
2562           Teton    WY  56039      3
2563           Uinta    WY  56041      5
2564        Washakie    WY  56043      2
2565          Weston    WY  56045      2

[2566 rows x 4 columns]

In [52]:
VacSites = VacSites_1.merge(fips_full, left_on=['FIPS'], right_on=['fips'])
VacSites.head(5)

VacSites


VacSites["County"] = VacSites["County_y"] + [', '] + VacSites["State_y"]
VacSites = VacSites[['County','Sites']]
#VacSites = VacSites.rename(columns={'County_y': 'County','State_y':'State'})

#print(VacSites[VacSites["State_y"] == 'AL'].to_string())
VacSites


County  Sites
0         Aleutians East Borough, AK      6
1     Aleutians West Census Area, AK      2
2         Anchorage Municipality, AK     38
3             Bethel Census Area, AK      1
4            Bristol Bay Borough, AK      1
...                              ...    ...
2561           Sweetwater County, WY      6
2562                Teton County, WY      3
2563                Uinta County, WY      5
2564             Washakie County, WY      2
2565               Weston County, WY      2

[2566 rows x 2 columns]

### Extra code to look for discrepancies

In [28]:

#sta = 'MD'
#df2 = df_sites[df_sites["State"] == sta]
#fips2 = fips[fips["State"] == sta]
#print(df2['County'].isin(fips2['County']).to_string())


#print(df_sites['County'].isin(foo['County']).to_string())

#df2 = df_sites[df_sites["State"] == sta]
#print(df2.to_string())

#fips2 = fips[fips["State"] == sta]
#print(fips2.to_string())

# Plotting

In [29]:
#ax1 = df_sites.plot.bar(x= 'State',y = 'offers_vaccine')

#ax1.set_ylim(0,250)

#df_AK = df_sites[df_sites['State'] == 'AK']

#df_AK.head(5).plot.bar(x='county', y='offers_vaccine', rot=0)
#fips_sites = fips.groupby(['State','County']).count()

#Reset index 
#fips_sites = fips_sites[['fips']].reset_index()


Save to csv

In [55]:
VacSites.to_csv('data/VacSites_Updated.csv')